In [1]:
import pandas as pd

In [2]:
dataset_name = 'sidra_tab1092_bovino'
raiz = '../database'

### Bronze

In [18]:
(pd.read_json('https://apisidra.ibge.gov.br/values/t/1092/n3/all/v/285/p/last%2094/c12716/115236/c18/992/c12529/118225')
 .to_json(f'{raiz}/bronze_{dataset_name}_uf.json')
 )

(pd.read_json('https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/285/p/last%2094/c12716/115236/c18/992/c12529/118225')
 .to_json(f'{raiz}/bronze_{dataset_name}_BR.json')
 )

### Prata

In [10]:
(pd.read_json(f'{raiz}/bronze_{dataset_name}_uf.json')
 .filter(['D1C', 'D3C', 'V'])
 .drop(0, axis = 0)
 .assign(**{'ano': lambda tri: tri['D3C'].str.extract('(\\d{4})').astype(int),
            'peso_kg': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'uf': lambda df: df['D1C'].astype(int),
            'fonte': 'Carcaça Bovina'
            })
 .dropna(subset='peso_kg')
 .groupby(['uf', 'ano', 'fonte'])
 .agg({'peso_kg': sum})
 #.to_parquet(f'{raiz}/prata_{dataset_name}.parquet')
 )

peso_kg
uf ano  fonte                     
28 2002 Carcaça Bovina  13886134.0
   2003 Carcaça Bovina  13603353.0
   2009 Carcaça Bovina  14338974.0
   2010 Carcaça Bovina  25217923.0
   2011 Carcaça Bovina  24914361.0
   2012 Carcaça Bovina  27684537.0
   2013 Carcaça Bovina  24777885.0
   2014 Carcaça Bovina  25388296.0
   2015 Carcaça Bovina  22749393.0
   2016 Carcaça Bovina  22341552.0
   2017 Carcaça Bovina  23839823.0
   2018 Carcaça Bovina  30905056.0
   2019 Carcaça Bovina  11455876.0
   2021 Carcaça Bovina  38786572.0
   2022 Carcaça Bovina  58272637.0
   2023 Carcaça Bovina  30642485.0

In [6]:
(pd.read_json(f'{raiz}/bronze_{dataset_name}_BR.json')
 .filter(['D1C', 'D3C', 'V'])
 .drop(0, axis = 0)
 .assign(**{'ano': lambda tri: tri['D3C'].str.extract('(\\d{4})').astype(int),
            'peso_kg': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'uf': lambda df: df['D1C'].astype(int),
            'fonte': 'Carcaça Bovina'
            })
 .query('ano >= 2000')
 .groupby(['uf', 'ano', 'fonte'])
 .agg({'peso_kg': sum})
 #.query('uf == 53')
 )

peso_kg
uf ano  fonte                     
1  2000 Carcaça Bovina  3899799609
   2001 Carcaça Bovina  4330277696
   2002 Carcaça Bovina  4699612954
   2003 Carcaça Bovina  4977213166
   2004 Carcaça Bovina  5906212281
   2005 Carcaça Bovina  6345811205
   2006 Carcaça Bovina  6886583120
   2007 Carcaça Bovina  7048994658
   2008 Carcaça Bovina  6621374461
   2009 Carcaça Bovina  6661632696
   2010 Carcaça Bovina  6977484368
   2011 Carcaça Bovina  6783536946
   2012 Carcaça Bovina  7351147177
   2013 Carcaça Bovina  8166720207
   2014 Carcaça Bovina  8063224819
   2015 Carcaça Bovina  7493435357
   2016 Carcaça Bovina  7358777695
   2017 Carcaça Bovina  7681537705
   2018 Carcaça Bovina  7989515631
   2019 Carcaça Bovina  8218851226
   2020 Carcaça Bovina  7824888302
   2021 Carcaça Bovina  7456261141
   2022 Carcaça Bovina  8012319797
   2023 Carcaça Bovina  4082151916

### Ouro

Peso total de carcaças dos bovinos abatidos (kg)

Aproveitamento da carcaça: 50%

Cálculo: ((Carcaça Bovina/aproveitamento%) * (Parâmetro_x%))/1000

UF e Brasil = t

NPK na forma de Carne bovina retirada do solo via animal encaminhado para abate (t)

In [4]:
aproveitamento = 50/100

parametro_N = 3/100
parametro_P = 0.8/100
parametro_K = 0.2/100

In [5]:
df = (pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet',
                      use_nullable_dtypes=True)
      .assign(**{
          'N': lambda df: ((df['peso_kg']/aproveitamento)*parametro_N)/1000,
          'P': lambda df: ((df['peso_kg']/aproveitamento)*parametro_P)/1000,
          'K': lambda df: ((df['peso_kg']/aproveitamento)*parametro_K)/1000
          })
      .reset_index()
      .melt(id_vars=['uf', 'ano', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .convert_dtypes()
      .set_index(['uf', 'ano', 'fonte'])
      )

#df.to_csv(f'{raiz}/ouro_{dataset_name}.csv')

df

elemento        valor
uf ano  fonte                               
11 2000 Carcaça Bovina        N     5872.623
   2001 Carcaça Bovina        N    7652.7219
   2002 Carcaça Bovina        N  12206.90538
   2003 Carcaça Bovina        N  12620.08572
   2004 Carcaça Bovina        N  14316.70368
...                         ...          ...
53 2010 Carcaça Bovina        K    17.887404
   2011 Carcaça Bovina        K    33.979116
   2012 Carcaça Bovina        K    35.954308
   2014 Carcaça Bovina        K    20.862764
   2015 Carcaça Bovina        K    77.469116

[1782 rows x 2 columns]